In [1]:
# Import libraries

import pandas as pd
import numpy as np
from datetime import datetime as dt
import pickle

# Preparing labels patient data

In [2]:
# Paths

in_path = '/ctao_shared_data/MCI_optum/Deep_learning/MCI/'   
ot_path = '../../Data/'

In [ ]:
# Load labels file which contains patient information : each row represents one patient
df = pd.read_csv(in_path + 'demo_adRelabel_fstAdReLabel_timeRelabel_80931_JF.csv')

# Print total number of patients
print("Total number of patients ", len (df)) 

In [ ]:
# Filtering patients younger than 40 years old at the first MCI diagnosis
age_filtered= df[df['age_dx_mci_1st']>=40]
age_filtered

In [ ]:
# Include only patients identified as Males or Females
age_filtered = age_filtered[(age_filtered['sex'] == 'M') | (age_filtered['sex'] == 'F')]
age_filtered 

In [ ]:
# Save a final list of eligible patients in .pkl file
# Number of patients: 80915

u_p_list = age_filtered.patid.unique()
with open(ot_path + 'MCI_p_list_final.pkl', 'wb') as f:
    pickle.dump(u_p_list.tolist(), f)


# Diagnoses

## Preprocess Diagnoses Data

In [ ]:
# Load diagnosis file : this file contains patient_id, claim_date, diagnosis code and ICD code flag
f = "final_diagnosis_all"
diagnosis_original = pd.read_csv(in_path+f+'.csv', delimiter = ',', low_memory = False)

In [ ]:
# Filter patients using the list of eligible patients
diag_filtered = diagnosis_original[diagnosis_original['patid'].isin(u_p_list)]

In [ ]:
# Convert wrong coded ICD10-CM to ICD9
# Codes starts with a digit is an ICD9 Code
a_is_digit = diag_filtered.diag.str[0].str.isdigit()
diag_filtered.loc[a_is_digit ,'icd_flag']= 9
diag_filtered= diag_filtered.drop_duplicates()

# Convert wrong coded ICD9 to ICD10-CM 
# Codes that don't start with V or E or first letter is not digit should be marked as ICD10-CM
dont_start_with_V_or_E = ~diag_filtered.diag.str.get(0).isin(['V','E']) & ~diag_filtered.diag.str[0].str.isdigit()
diag_filtered.loc[dont_start_with_V_or_E,'icd_flag']= 10
diag_filtered = diag_filtered.drop_duplicates()

## Mapping to Phecodes

In [ ]:
# Load mapping file from diagnosis ICD codes to Phecodes
phe_file = "raw_to_phewas.txt"

# Preprocess Phecodes file
f_loaded = pd.read_csv(in_path + phe_file , delimiter = '\t',low_memory = False)
f_loaded = f_loaded[f_loaded["diagnosis_type"]!= 'ICD10_CA']
f_loaded['diagnosis_code']= (f_loaded['diagnosis_code']).astype(str).str.replace('.', '')
f_loaded['phe_icd']= (f_loaded['phe_icd']).astype(str).str.replace('.', '')

In [ ]:
# Merging Original codes and phecodes
merged_codes = pd.merge(diag_filtered, f_loaded, how = 'left',left_on=['diag','icd_flag'], right_on=['diagnosis_code','diagnosis_type'])
merged_codes = merged_codes.drop_duplicates()
merged_codes = merged_codes.dropna()

## Save final mapped diagnosis file

In [ ]:
col = ['patid', 'fst_dt', 'phecode']
final_diagnosis_phewas = (
    merged_codes[col]
    .drop_duplicates()
    .assign(
        aggregation_window_id=lambda df: df.patid.astype(str) + '_' + df.fst_dt.astype(str),
        modified_diag=lambda df: 'D_' + df.phecode.astype(str)
    )[['patid', 'fst_dt', 'aggregation_window_id', 'modified_diag']]
)
final_diagnosis_phewas.to_csv(ot_path + 'phewas_diagnosis_dataset_final.csv', index=False)

# Procedures

## Preprocess Procedures Data

In [ ]:
# Load procedures file : this file contains patient_id, claim_date, procedure code and ICD code flag
f = "final_proc_all"
procedures_original =  pd.read_csv(in_path + f + '.csv', delimiter = ',', low_memory = False)

In [ ]:
# Filter patients using the list of eligible patients
procedures_filtered = procedures_original[procedures_original['patid'].isin(u_p_list)]

In [ ]:
# Fix Incorrect ICD-10 Codes
# Filter the DataFrame to include only rows where the length of 'proc' is >= 3 and not equal to 6
procedures_filtered = procedures_filtered[
    procedures_filtered.proc.str.len().ge(3) & procedures_filtered.proc.str.len().ne(6)
].assign(
    # Update the 'icd_flag' column: set it to 10.0 for rows where the length of 'proc' is exactly 7,
    # otherwise retain the original value of 'icd_flag'
    icd_flag=lambda df: np.where(df.proc.str.len() == 7, 10.0, df.icd_flag)
)

## Converting to CCS  (Clinical Classification Software) Codes

In [ ]:
# Working on ICD9-CM

# Step 1: Filter Procedures for ICD-9-CM Codes
proc_icd_9_cm = procedures_filtered.loc[(procedures_filtered['proc'].str.len() == 4) | (procedures_filtered['proc'].str.len() == 3) ]
proc_icd_9_cm
proc_icd_9_cm.icd_flag = 9.0

# Step 2: Extract unique ICD-9-CM Codes
col = ['proc', 'icd_flag']
unique_proc_icd_9_cm = proc_icd_9_cm[col].drop_duplicates()
unique_proc_icd_9_cm

# Step 3: Load and Preprocess CCS ICD-9-CM Mapping File
ccs_icd9_file = "$prref 2015.csv"
ccs_icd9_file_loaded = pd.read_csv(in_path + ccs_icd9_file , delimiter = ',',low_memory = False)
ccs_icd9_file_loaded.rename(columns=lambda x: x.strip("'"), inplace=True)
ccs_icd9_file_loaded= ccs_icd9_file_loaded.replace("  '", '', regex=True)
ccs_icd9_file_loaded= ccs_icd9_file_loaded.replace(" '", '', regex=True)
ccs_icd9_file_loaded= ccs_icd9_file_loaded.replace("'", '', regex=True)

# Step 4: Merge ICD-9-CM Codes with CCS Data
merged_icd_9_cm = unique_proc_icd_9_cm.merge(ccs_icd9_file_loaded, how = 'left',left_on=['proc'], right_on=['ICD-9-CM CODE'])
merged_icd_9_cm = merged_icd_9_cm.dropna() 
merged_icd_9_cm

# Step 5: Creates a dictionary that maps each ICD-9-CM procedure code to its corresponding CCS
Final_dict = dict(zip(merged_icd_9_cm['proc'], merged_icd_9_cm['CCS CATEGORY']))

In [ ]:
# Working on ICD-10-PCS

# Remaining#1: Filter Out ICD-9-CM Procedures
# Exclude rows already classified as ICD-9-CM codes from the original dataset
procedures_updated1 = procedures_filtered.loc[procedures_filtered.index.difference(proc_icd_9_cm.index)]

# Step 1: Filter Procedures for ICD-10-PCS
proc_icd_10_pcs = procedures_updated1[(procedures_updated1.icd_flag == 10.0) & (procedures_updated1.proc.str.len() == 7)]

# Step 2: Extract Unique ICD-10-PCS Codes
col = ['proc', 'icd_flag']
unique_proc_icd_10_pcs = proc_icd_10_pcs[col].drop_duplicates()

# Step 3: Load and Preprocess CCS ICD-10 Mapping File
ccs_icd10_file = "ccs_pr_icd10pcs_2019_2.csv"
ccs_icd10_file_loaded = pd.read_csv(in_path + ccs_icd10_file , delimiter = ',',low_memory = False)
ccs_icd10_file_loaded.rename(columns=lambda x: x.strip("'"), inplace=True)
ccs_icd10_file_loaded = ccs_icd10_file_loaded.replace("'", '', regex=True)

# Step 4: Merge ICD-10-PCS Codes with Classification Data
merged_icd_10 = unique_proc_icd_10_pcs.merge(ccs_icd10_file_loaded, how = 'left',left_on=['proc'], right_on=['ICD-10-PCS CODE'])
merged_icd_10 = merged_icd_10.dropna()

# Step 5: Update Final Dictionary - Integrate ICD-10-PCS classifications into the final mapping dictionary
icd_10_dict = dict(zip(merged_icd_10['proc'], merged_icd_10['CCS CATEGORY']))
Final_dict.update(icd_10_dict)

In [ ]:
# Working on HCPCS Level I and HCPCS Level II Procedure Codes

# Remaining#2 - Exclude ICD-9-CM and ICD-10-PCS Codes
procedures_updated2 = procedures_updated1.loc[procedures_updated1.index.difference(proc_icd_10_pcs.index)]


#Step 1: Extract Unique HCPCS Codes
col = ['proc', 'icd_flag']
unique_proc_HCPCS = procedures_updated2[col].drop_duplicates()

#Step 2: Load and Parse the HCPCS Mapping File
ccs_HCPCS_file = "CCS_services_procedure_v2019-2022_keep_updated.csv"
final_map_all = pd.read_csv(in_path + ccs_HCPCS_file ,low_memory = False)
#Split the Code Range into Start and End
final_map_all[["code1", "code2"]] = final_map_all["Code Range"].str.split("-", expand=True)
map_dict = final_map_all[['CCS','code1','code2']].to_dict('records')

#Step 3: Match HCPCS Codes to Ranges
proc_list = unique_proc_HCPCS.proc.unique().tolist()
for proc_code in set(proc_list):
    for item in map_dict:
        for k in item.keys():
            if k =='code1': 
                code1 = item.get(k)
            if k =='code2':
                code2 = item.get(k) 
        if (proc_code == code1) or (proc_code == code2) :
            item.setdefault('proc_cd',[]).append(proc_code)
            break
        elif (proc_code > code1) and (proc_code < code2) :
            item.setdefault('proc_cd',[]).append(proc_code)
            break

#Step 4: Create a DataFrame for Mapped Codes
new_map = pd.DataFrame.from_dict(map_dict)
#Filter and Explode Matched Codes
matched_new_map = new_map[~new_map.proc_cd.isnull()]
matched_new_map_explode = matched_new_map.explode('proc_cd').drop_duplicates(ignore_index=True)
matched_new_map_explode['CCS'] = matched_new_map_explode['CCS'].astype(str) 

#Step 5: Merge HCPCS Codes with Classification Data
merged_HCPCS = unique_proc_HCPCS.merge(matched_new_map_explode, how = 'left',left_on=['proc'], right_on=['proc_cd'])
merged_HCPCS = merged_HCPCS.dropna() 


#Adding HCPCS Codes to the Final Dictionary
HCPCS_dict = dict(zip(merged_HCPCS['proc'], merged_HCPCS['CCS']))
Final_dict.update(HCPCS_dict)

## Remapping original codes to CCS and save final mapped procedure file

In [ ]:
procedures_filtered["proc"] = procedures_filtered["proc"].map(Final_dict)
procedures_filtered = procedures_filtered.dropna()
procedures_filtered['aggregation_window_id'] = procedures_filtered.patid.astype(str) + '_' + procedures_filtered.fst_dt.astype(str)
procedures_filtered['mod_proc_code'] = 'P_' + procedures_filtered['proc'].astype(str)

cl = ['patid', 'fst_dt', 'aggregation_window_id', 'mod_proc_code']
procedures_filtered[cl].to_csv(ot_path + 'mod_proc_dataset_final.csv', index=False)

# Medications

## Preprocess medications

In [ ]:
# Load drugs file
f ='final_drug_all'
drugs_original = pd.read_csv(in_path + f + '.csv',  delimiter = ',', low_memory = False)

# Filter patients using the list of eligible patients
drugs_filtered = drugs_original[drugs_original['patid'].isin(u_p_list)]

In [ ]:
# Load the NDC (National Drug Code) data from a tab-delimited file
ndc = pd.read_csv(in_path + 'lu_ndc.csv',delimiter = '\t', header=1, low_memory=False)

# Select only the relevant columns: 'NDC' (National Drug Code) and 'GNRC_NM' (Generic Name)
ndc = ndc[['NDC','GNRC_NM']].drop_duplicates().reset_index(drop=True)
ndc = ndc.dropna()

## Maping NDC to generic Names

In [ ]:
# Create a Dictionary for NDC to Generic Name Mapping
ndc_gnrc_nm = dict(zip(ndc['NDC'], ndc['GNRC_NM'])) 

gnrc_nm_list = []
for ndc in drugs_filtered['ndc']:
    if ndc in ndc_gnrc_nm:
        gnrc_nm_list.append(ndc_gnrc_nm[ndc])
    else:
        gnrc_nm_list.append(np.nan)
# Add the Generic Names to the Drug DataFrame
drugs_filtered['gnrc_nm']= gnrc_nm_list
drugs_filtered = drugs_filtered.dropna()

## Save final mapped medications file

In [ ]:
drugs_filtered['encounter_id'] = drugs_filtered.patid.astype(str)+ '_'+ drugs_filtered.fst_dt.astype(str)
mod_gen_drug_list = []
for gnrc_nm in drug.gnrc_nm:
    modified_name = 'M_' + gnrc_nm.lower().replace('.', '')
    mod_gen_drug_list.append(modified_name)
drugs_filtered['mod_gen_drug'] = mod_gen_drug_list

cl = ['patid',  'fst_dt', 'aggregation_window_id', 'mod_gen_drug']
drugs_filtered = drugs_filtered[cl]
drugs_filtered.to_csv(ot_path + 'mod_drug_dataset_final.csv',index=False)

# Grouping claims into encounters

In [ ]:
cl = ['patid','fst_dt','aggregation_window_id']


df1 = pd.read_csv(ot_path + 'phewas_diagnosis_dataset_final.csv' , delimiter = ',', low_memory = False, usecols=cl)
df2 = pd.read_csv(ot_path + 'mod_proc_dataset_final.csv', delimiter = ',', low_memory = False, usecols=cl)
df3 = pd.read_csv(ot_path + 'mod_drug_dataset_final.csv', delimiter = ',', low_memory = False, usecols=cl)

df_concat = pd.concat([df1, df2, df3], axis=0)
clm_df  = df_concat.drop_duplicates()
   
clm_df = clm_df[['aggregation_window_id', 'patid', 'fst_dt']]
clm_df = clm_df.sort_values(by=['aggregation_window_id']).reset_index(drop=True)
clm_df.to_csv(ot_path + 'dict_encounter_Dataset_final.csv',index=False)